In [20]:
#from cnn_model import *
%run cnn_model.ipynb
import sys
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"






model = cnn_model_1_layer()
train_step, accuracy = model.run_cnn()

train_accuracy = 0.0



#initialize
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


#train for 1000 times
for i in range(1000):
    batch = mnist.train.next_batch(100)
    #跑訓練集，利用optimizer將結果導到較好的地方
    train_step.run(session=sess, feed_dict={model.img_placeholder: batch[0],
                                            model.label_placeholder: batch[1], model.keep_prob: 0.7})
    
    
    if i%10 == 0:
        #輸出準確度
        train_accuracy = accuracy.eval(session=sess, feed_dict={model.img_placeholder:batch[0], 
                                        model.label_placeholder: batch[1], model.keep_prob: 1.0})
        
        print("step {}, training accuracy {:}".format(i, train_accuracy))
        
           
    

print("Accuracy: ", sess.run(accuracy, feed_dict={model.img_placeholder: mnist.test.images,
                                model.label_placeholder: mnist.test.labels, model.keep_prob: 1.0}))



# Saving

export_dir =  "my_net/save_net.ckpt"
# saver = tf.train.Saver()
saver = tf.train.Saver({
            "W_conv1": model.W_conv1, 
            "W_conv2": model.W_conv2, 
            "b_conv1": model.b_conv1, 
            "b_conv2": model.b_conv2, 
            "W_fc1": model.W_fc1, 
            "W_fc2": model.W_fc2, 
            "b_fc1": model.b_fc1, 
            "b_fc2": model.b_fc2
        })
sess.run(init)
save_path = saver.save(sess, export_dir)
#save_path = saver.save(sess, export_dir, global_step = 200)
print("Model saved in path: ", save_path)




Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.18000000715255737
step 10, training accuracy 0.3799999952316284
step 20, training accuracy 0.4099999964237213
step 30, training accuracy 0.5199999809265137
step 40, training accuracy 0.5400000214576721
step 50, training accuracy 0.49000000953674316
step 60, training accuracy 0.44999998807907104
step 70, training accuracy 0.6200000047683716
step 80, training accuracy 0.46000000834465027
step 90, training accuracy 0.6100000143051147
step 100, training accuracy 0.6399999856948853
step 110, training accuracy 0.6200000047683716
step 120, training accuracy 0.6700000166893005
step 130, training accuracy 0.6899999976158142
step 140, training accuracy 0.5699999928474426
step 150, training accuracy 0.7300000190734863
step 160, training accuracy 0.7300000190734863
step 170, trai

AttributeError: 'cnn_model_1_layer' object has no attribute 'W_conv2'